In [1]:
!python --version

Python 3.7.9


In [2]:
# !pip install -q -U pip
# !pip install -q numpy
# !pip install -q pandas
# !pip install -q ckiptagger
# !pip install -q tqdm
# !pip install -q tensorflow==1.14.0
# !pip install -q ipywidgets

In [3]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

In [4]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [5]:
df_train

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍
...,...,...,...
1795,1795,遊戲,LOL：麻辣香鍋韓服Rank不合成打野刀？電刀巨魔新套路連勝中
1796,1796,遊戲,穩住，我們能贏！因為我們擁有這種強大的力量
1797,1797,遊戲,騰訊是怎樣毀掉《鬥戰神》這款可能成神的作品的？
1798,1798,遊戲,LOL你不知道的黑科技打法！


In [6]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [7]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [8]:
ws = WS('./data/')
pos = POS('./data/')

In [11]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [12]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [11]:
train_title_cuts[120]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

# Bag of Words (BOW)

In [13]:
word2index = {}
index2word = {}
# 產生字與index對應的關係
n = 0
for index in train_title_cuts:
    for word, flag in train_title_cuts[index]:
        if word in word2index:
            continue
        word2index[word] = n
        index2word[n] = word
        n += 1
        

In [14]:
word2index['溫暖']

1520

In [15]:
index2word[1520]

'溫暖'

In [18]:
def get_bow_vector(pairs, word2index):
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index:
            vector[word2index[word]] += 1
    return vector

In [19]:
get_bow_vector(train_title_cuts[120], word2index)

array([0., 0., 0., ..., 0., 0., 0.])

# 排除較無意義的詞性

In [20]:
pos_analysis = {}
for _, pairs in train_title_cuts.items():
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:100])
    print('=======================')

Nb : ['弗格森', '朱', '溫格', '亞特拉斯賽', '埃姆雷', '安迪爾森', '里皮', '余承東', '游玄德', 'J羅', '萊萬特', '小詹皇', 'P60八核讓', '璞麗', '殺里', '盧克肖', '查理芒格', '斐濟', '馬內', '衛報', '切沃', '天貓', '卡帥', '聯發科', '世界杯', '克羅托內 ', '新游', '龔方雄', '喬丹', '浦發', '奧拉朱旺遠', '張本智和', '陶華碧', '詹', '納米 華', '郎平', '慧商', '卡拉斯科', '微眾', '伊藤美誠', '娜迦王海伯倫', '賈躍亭', '德安東尼', '蘇群', '武當', '奧恰洛夫', '阿迪薩亞', '東決', '朱嘯虎', '紫鑫', '勇士隊', '約翰塞納', '海富通', '郭春林', '曼聯', '恆生', '梵', '億航', '·龜茲', '何享健', '凱爾特', '湯米', '伯克希爾', '馬夏爾', '李謹詺', '塔特姆', '泰倫', '伊亞', '高傭', '科爾', '唐汪鎮', '馬化騰', '何雯娜', '大豐', '內馬爾', '中金', '瓦基弗', '泰拳王西', '李白界', '林員', '安利', '圍甲', '騰訊', '·瞻', '石川佳純', 'Dota2', '景泰石', '富力', '斯諾克', '大美', '阿里雲', '大神', '四阿哥', '丁寧', '丘誠', '加威廉', '皮爾洛', '迪士尼', '馬克思', '王楚欽']
VC : ['碰到', '靠近', '獨造', '防治', '扒出', '豪取', '春栽', '命懸', '打死', '直播', '摘', '調侃', '兌換', '掛', '屠殺', '毀滅', '關掉', '揭密', '收穫', '控制', '放棄', '採摘', '脫離', '攻略', '丟', '娶', '集訓', '添脫', '踐行', '駕馭', '觀賞', '治療', '完虐', '接受', '打不過', '釋放', '吃好', '喜迎', '要', '鋪開', '廢', '看待', '提升', '嘲諷', '傳用', '領取', '起到', '收', '虛擬', '打', '取得', '列', '教會'

|         Type        |     Description    |
|:-------------------:|:------------------:|
| A                   | 非謂形容詞         |
| Caa                 | 對等連接詞         |
| Cab                 | 連接詞，如：等等   |
| Cba                 | 連接詞，如：的話   |
| Cbb                 | 關聯連接詞         |
| D                   | 副詞               |
| Da                  | 數量副詞           |
| Dfa                 | 動詞前程度副詞     |
| Dfb                 | 動詞後程度副詞     |
| Di                  | 時態標記           |
| Dk                  | 句副詞             |
| DM                  | 定量式             |
| I                   | 感嘆詞             |
| Na                  | 普通名詞           |
| Nb                  | 專有名詞           |
| Nc                  | 地方詞             |
| Ncd                 | 位置詞             |
| Nd                  | 時間詞             |
| Nep                 | 指代定詞           |
| Neqa                | 數量定詞           |
| Neqb                | 後置數量定詞       |
| Nes                 | 特指定詞           |
| Neu                 | 數詞定詞           |
| Nf                  | 量詞               |
| Ng                  | 後置詞             |
| Nh                  | 代名詞             |
| Nv                  | 名物化動詞         |
| P                   | 介詞               |
| T                   | 語助詞             |
| VA                  | 動作不及物動詞     |
| VAC                 | 動作使動動詞       |
| VB                  | 動作類及物動詞     |
| VC                  | 動作及物動詞       |
| VCL                 | 動作接地方賓語動詞 |
| VD                  | 雙賓動詞           |
| VF                  | 動作謂賓動詞       |
| VE                  | 動作句賓動詞       |
| VG                  | 分類動詞           |
| VH                  | 狀態不及物動詞     |
| VHC                 | 狀態使動動詞       |
| VI                  | 狀態類及物動詞     |
| VJ                  | 狀態及物動詞       |
| VK                  | 狀態句賓動詞       |
| VL                  | 狀態謂賓動詞       |
| V_2                 | 有                 |
|                     |                    |
| DE                  | 的之得地           |
| SHI                 | 是                 |
| FW                  | 外文               |
|                     |                    |
| COLONCATEGORY       | 冒號               |
| COMMACATEGORY       | 逗號               |
| DASHCATEGORY        | 破折號             |
| DOTCATEGORY         | 點號               |
| ETCCATEGORY         | 刪節號             |
| EXCLAMATIONCATEGORY | 驚嘆號             |
| PARENTHESISCATEGORY | 括號               |
| PAUSECATEGORY       | 頓號               |
| PERIODCATEGORY      | 句號               |
| QUESTIONCATEGORY    | 問號               |
| SEMICOLONCATEGORY   | 分號               |
| SPCHANGECATEGORY    | 雙直線             |
| WHITESPACE          | 空白               |

In [21]:
def get_bow_vector_with_selection(pairs, word2index):
    excluded_flags = [
        # 根據以上列舉出來的文字以及詞性表，請列出想要排除的詞性
        'Nh', 'Nep', 'Nes', 'DE', 'T', 'P', 'V_2', 'SHI',
        'Dfa', 'Dfb', 'Da', 'Di', 'Dk',
        'Caa', 'Cab', 'Cba', 'Cbb',
        'COLONCATEGORY', 'COMMACATEGORY', 'DASHCATEGORY', 'DOTCATEGORY', 'ETCCATEGORY', 'EXCLAMATIONCATEGORY',
        'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'PERIODCATEGORY', 'QUESTIONCATEGORY', 'SEMICOLONCATEGORY',
        'SPCHANGECATEGORY', 'WHITESPACE'
    ]
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

# Cosine Similarity

In [22]:
def cosine_similarity(bow1, bow2):
    len_bow1 = np.sqrt(np.sum(np.square(bow1)))
    len_bow2 = np.sqrt(np.sum(np.square(bow2)))
    similarity = np.sum(bow1*bow2) / (len_bow1 * len_bow2)
    return similarity

In [23]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882797784893

In [24]:
train_title_cuts[100]

[('山東', 'Nc'),
 ('魯能', 'Nb'),
 ('有沒有', 'D'),
 ('可能', 'D'),
 ('拿到', 'VC'),
 ('今年', 'Nd'),
 ('的', 'DE'),
 ('中', 'A'),
 ('超', 'A'),
 ('冠軍', 'Na'),
 ('？', 'QUESTIONCATEGORY')]

In [25]:
train_title_cuts[130]

[('NBA', 'Nb'),
 ('和', 'Caa'),
 ('CBA', 'FW'),
 ('差距', 'Na'),
 ('在', 'P'),
 ('哪裡', 'Ncd'),
 ('？', 'QUESTIONCATEGORY'),
 ('6', 'Neu'),
 ('張', 'Nf'),
 ('圖', 'VF'),
 ('一目瞭然', 'VH'),
 ('！', 'EXCLAMATIONCATEGORY')]

# Group mean vector

In [26]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
group_mean_vector

{'體育': array([0.04      , 0.01333333, 0.00333333, ..., 0.        , 0.        ,
        0.        ]),
 '財經': array([0., 0., 0., ..., 0., 0., 0.]),
 '科技': array([0., 0., 0., ..., 0., 0., 0.]),
 '旅遊': array([0., 0., 0., ..., 0., 0., 0.]),
 '農業': array([0., 0., 0., ..., 0., 0., 0.]),
 '遊戲': array([0.        , 0.        , 0.        , ..., 0.00333333, 0.00333333,
        0.00333333])}

# Group mean vector: 測試

In [27]:
classification = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [28]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 73, '遊戲': 8, '財經': 6, '旅遊': 6, '農業': 6, '科技': 3})
predict 財經 :  Counter({'財經': 67, '科技': 18, '體育': 6, '旅遊': 6, '遊戲': 6, '農業': 5})
predict 科技 :  Counter({'科技': 62, '財經': 13, '體育': 10, '農業': 6, '遊戲': 6, '旅遊': 2})
predict 旅遊 :  Counter({'旅遊': 70, '農業': 11, '財經': 6, '科技': 3, '遊戲': 2})
predict 農業 :  Counter({'農業': 70, '旅遊': 9, '科技': 6, '體育': 5, '財經': 4, '遊戲': 3})
predict 遊戲 :  Counter({'遊戲': 75, '科技': 7, '旅遊': 5, '體育': 4, '財經': 4, '農業': 1})
